In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import matplotlib.pyplot as plt
from scipy import stats
from scipy import cluster

In [2]:
n = 1000
beta01, beta11 = 5,-3
beta02, beta12 = 2, 4

#set up regression mixture
x1 = np.random.uniform(0, 10, size=400)
x2 = np.random.uniform(0, 10, size=600)

y1 = beta01 + beta11*x1 + np.random.normal(scale=5.0, size=400)
y2 = beta02 + beta12*x2 + np.random.normal(scale=4.0,size=600)

X = np.concatenate([x1, x2])
Y = np.concatenate([y1, y2])


#set up 2 component mixture
a1 = np.random.normal(2, 5, size=600)
a2 = np.random.normal(5, 3, size=400)
a = np.concatenate([a1,a2])

In [3]:
def e_step(y,x,params): 
    y, x = np.array(y), np.array(x)
    nobs, k = x.shape
    weights = []
    for param in params:

        sigma = param[-1]
        beta = np.tile(param[1:-1],nobs).reshape(nobs, k)
        mean = (beta*x).sum(axis=1)
        weights.append( stats.norm.pdf(y, loc=mean, scale=sigma)*param[0] )
        
    #update loop variables
    weights = np.array(weights).transpose()
    #denom = np.repeat( 1+np.exp(weights).sum(axis=1), len(params) ).reshape(nobs,len(params))
    denom = np.repeat(weights.sum(axis=1), len(params) ).reshape(nobs,len(params))
    weights = weights/denom
    return weights, np.log(denom[:,0])
        
    
def m_step(y,x,weights):
    y, x, weights = np.array(y), np.array(x), np.array(weights)
    nobs, k = x.shape
    params, se, err = [], [], 0

    for w in weights.transpose():
        
        lamb = w.mean()
        lamb_se = w.std()

        #beta
        w_mat = np.diag(w)
        xx_mat = np.linalg.inv( x.transpose().dot( w_mat).dot(x) )
        beta = xx_mat.dot(x.transpose().dot(w_mat)).dot(y)
        
        #sigma
        mu = np.tile(beta, nobs).reshape(nobs, k)*x
        weighted_err = w*(y - mu.sum(axis=1))**2
        sigma =  (weighted_err.sum()/w.sum())**.5

        #add component
        comp_param =np.concatenate(([lamb],beta,[sigma]))
        params.append(comp_param)

        #beta_se
        beta_se = (np.diagonal(xx_mat*sigma**2))**.5
        comp_se = np.concatenate(([lamb_se],beta_se))
        se.append(comp_se)

        #SSR
        err = err+weighted_err
    return np.array(params), np.array(se), 1-err.mean()/y.var()


def gen_weights(y,ncomp):
    c,labels = cluster.vq.kmeans2(y,ncomp)
    return np.array(pd.get_dummies(labels))


def estimate(y,x,ncomp):
    e = gen_weights(y,ncomp)
    m = None
    for i in range(20):
        m,se,r2 = m_step(y,x,e)
        e,ll = e_step(y,x,m)
    return m, se, r2, y, x, ncomp, ll


m, se, r2, y, x, ncomp, ll = estimate(Y, sm.add_constant(X), 1)

In [4]:
def write_table(fname, estimates, labels=('y',None)):
    
    #unpack relevant information
    params, se, r2, y, x, ncomp, ll = estimates
    nobs, k = x.shape
    ylabel, xlabel = labels
    
    if xlabel == None:
        xlabel =[]
        for i in range(k):
            xlabel.append('x%s'%i)
            
    assert (k == len(xlabel)) 
    
    f = open(fname, "w+")
    
    f.write(('\\begin{center}  \n'+
            '\\begin{tabular}{lclc} \n'+
            '\\toprule \n'+
            '\\textbf{Dep. Variable:} & %s & \\textbf{  R-squared: } &  %s \\\\ \n'%(ylabel, np.round(r2,3))  ))
    
    f.write(('\\textbf{No. Observations:} & %s & \\textbf{ Log-Likelihood:} & %s \\\\ \n'%(nobs,
                                                                                           np.round(ll.sum(),1))+
            '\end{tabular} \n'))
    
    
    f.write('\n\\begin{tabular}{lcccc} \n')
    for comp in range(ncomp):
        f.write('\\toprule \n')
        f.write('\\textbf{Regime %s} & \\textbf{est} & \\textbf{std err} &'%(1+comp)+ 
                '\\textbf{t} & \\textbf{P $>$ $|$ t $|$} \\\\ \n')
        f.write('\\bottomrule \\\\ \n')
        
        #isolate params
        comp_params = params[comp]
        comp_se = se[comp]
        comp_t = comp_params[:-1]/comp_se
        comp_p = 1 - stats.t.cdf(np.abs(comp_t),df=(nobs-k)) + stats.t.cdf(-np.abs(comp_t),df=(nobs-k))
        
        #round everything
        comp_params = np.round(comp_params,5)
        comp_se = np.round(comp_se,5)
        comp_t = np.round(comp_t,5)
        comp_p = np.round(comp_p,5)
        
        lamb = comp_params[0]
        #lamb_se, lamb_t, lamb_p = comp_params[0], comp_se[0], comp_t[0], comp_p[0]
        beta, beta_se, beta_t, beta_p = comp_params[1:-1], comp_se[1:], comp_t[1:], comp_p[1:]
        sigma = comp_params[-1]
        
        if ncomp > 1:
            f.write('\\textbf{lambda} & %s  & & & \\\\ \\\\ \n'%(lamb) )
        
        for i in range(k):
            f.write('\\textbf{%s} & %s & (%s) & %s & %s \\\\ \\\\ \n'%(xlabel[i],beta[i],beta_se[i],
                                                                             beta_t[i],beta_p[i]) )
        
        f.write('\\textbf{sigma} & %s &  & & \\\\ \\\\ \n'%(sigma) )
    
    f.write('\end{tabular} \n'+
            '\end{center}\n')
    f.close()
    
    #print output
    f = open(fname, "r")
    print f.read()
    f.close()

In [10]:
reg1 = pd.read_csv('data/clean_milk1.csv')
print reg1.columns

#variables names
lmilk = ['LWW']
auct_key = ['YEAR','MONTH','DAY','SYSTEM','FMOZONE']
lcts = ['LFMO','LGAS','LPOPUL','LQWW','LMEALS']
dummies = ['COOLER','ESC']
fekeys = ['1A', '3', '6', '7', '9']

lags = 3
lagkeys = [l+str(i) for l in ['LWW_min','LWW_max'] for i in range(1,1+lags)]

bid_key = auct_key + ['VENDOR'] + ['COUNTY']
covariates = lcts + dummies + fekeys
hist = ['INC'] + lagkeys

Index([u'Unnamed: 0', u'YEAR', u'MONTH', u'DAY', u'SYSTEM', u'FMOZONE',
       u'VENDOR', u'COUNTY', u'LWW', u'LFMO', u'LGAS', u'LPOPUL', u'LQWW',
       u'LMEALS', u'COOLER', u'ESC', u'1A', u'3', u'6', u'7', u'9', u'INC',
       u'LWW_min1', u'LWW_min2', u'LWW_min3', u'LWW_max1', u'LWW_max2',
       u'LWW_max3'],
      dtype='object')


In [11]:
est0 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates]),1)
write_table('results/ols_results.tex', est0, labels=('LWW', ['const']+covariates))

\begin{center}  
\begin{tabular}{lclc} 
\toprule 
\textbf{Dep. Variable:} & LWW & \textbf{  R-squared: } &  0.185 \\ 
\textbf{No. Observations:} & 3665 & \textbf{ Log-Likelihood:} & 4073.6 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\toprule 
\textbf{Regime 1} & \textbf{est} & \textbf{std err} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\bottomrule \\ 
\textbf{const} & -2.26908 & (0.072) & -31.51664 & 0.0 \\ \\ 
\textbf{LFMO} & 0.20991 & (0.02617) & 8.02093 & 0.0 \\ \\ 
\textbf{LGAS} & 0.00918 & (0.00409) & 2.2428 & 0.02497 \\ \\ 
\textbf{LPOPUL} & 0.02545 & (0.00315) & 8.09203 & 0.0 \\ \\ 
\textbf{LQWW} & -0.00012 & (0.00204) & -0.05959 & 0.95248 \\ \\ 
\textbf{LMEALS} & -0.02269 & (0.00306) & -7.40151 & 0.0 \\ \\ 
\textbf{COOLER} & 0.01777 & (0.00296) & 6.00939 & 0.0 \\ \\ 
\textbf{ESC} & -0.01976 & (0.00279) & -7.07503 & 0.0 \\ \\ 
\textbf{1A} & -0.02458 & (0.01893) & -1.29838 & 0.19424 \\ \\ 
\textbf{3} & -0.06477 & (0.0042) & -15.41156 & 0.0 \\ \\ 
\textbf{6} & -0.05256 & (0.00803)

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in divide


In [13]:
est1 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates + hist]),1)
write_table('results/hist_results.tex', est1, labels=('LWW', ['const']+covariates + hist))

\begin{center}  
\begin{tabular}{lclc} 
\toprule 
\textbf{Dep. Variable:} & LWW & \textbf{  R-squared: } &  0.211 \\ 
\textbf{No. Observations:} & 3665 & \textbf{ Log-Likelihood:} & 4134.1 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\toprule 
\textbf{Regime 1} & \textbf{est} & \textbf{std err} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\bottomrule \\ 
\textbf{const} & -1.68047 & (0.09641) & -17.43104 & 0.0 \\ \\ 
\textbf{LFMO} & 0.15998 & (0.0263) & 6.08293 & 0.0 \\ \\ 
\textbf{LGAS} & 0.01015 & (0.00406) & 2.50293 & 0.01236 \\ \\ 
\textbf{LPOPUL} & 0.02095 & (0.00314) & 6.67031 & 0.0 \\ \\ 
\textbf{LQWW} & 0.0016 & (0.00204) & 0.78401 & 0.43309 \\ \\ 
\textbf{LMEALS} & -0.02066 & (0.00304) & -6.80183 & 0.0 \\ \\ 
\textbf{COOLER} & 0.01863 & (0.00293) & 6.35513 & 0.0 \\ \\ 
\textbf{ESC} & -0.01905 & (0.00275) & -6.91821 & 0.0 \\ \\ 
\textbf{1A} & -0.01497 & (0.01869) & -0.8011 & 0.42313 \\ \\ 
\textbf{3} & -0.06531 & (0.00414) & -15.77324 & 0.0 \\ \\ 
\textbf{6} & -0.05735 & (0.00794) 

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in divide


In [14]:
est2 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates]),2)
write_table('results/prelim_results.tex', est2, labels=('LWW', ['const']+covariates))

\begin{center}  
\begin{tabular}{lclc} 
\toprule 
\textbf{Dep. Variable:} & LWW & \textbf{  R-squared: } &  0.439 \\ 
\textbf{No. Observations:} & 3665 & \textbf{ Log-Likelihood:} & 4401.4 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\toprule 
\textbf{Regime 1} & \textbf{est} & \textbf{std err} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\bottomrule \\ 
\textbf{lambda} & 0.6813  & & & \\ \\ 
\textbf{const} & -2.14093 & (0.0836) & -25.60835 & 0.0 \\ \\ 
\textbf{LFMO} & 0.19306 & (0.03017) & 6.39825 & 0.0 \\ \\ 
\textbf{LGAS} & -0.00927 & (0.00471) & -1.9654 & 0.04944 \\ \\ 
\textbf{LPOPUL} & 0.03933 & (0.00406) & 9.69727 & 0.0 \\ \\ 
\textbf{LQWW} & 0.00104 & (0.00229) & 0.45271 & 0.65078 \\ \\ 
\textbf{LMEALS} & -0.03515 & (0.00394) & -8.92843 & 0.0 \\ \\ 
\textbf{COOLER} & 0.01836 & (0.00342) & 5.37041 & 0.0 \\ \\ 
\textbf{ESC} & -0.01932 & (0.00329) & -5.87378 & 0.0 \\ \\ 
\textbf{1A} & -0.0401 & (0.02407) & -1.66591 & 0.09582 \\ \\ 
\textbf{3} & -0.02197 & (0.00478) & -4.5951 & 0.0 \\

In [15]:
test1 = estimate(Y, sm.add_constant(X), 2)
test2 = estimate(Y, sm.add_constant(X), 1)

def nonnested_test(model1,model2):
    """test for non nested models quang vuong"""
    
    params1, se1, r21, y1, x1, ncomp1, ll1 = model1
    params2, se2, r22, y2, x2, ncomp2, ll2 = model2
    nobs, k = x1.shape
    
    k1 = params1.shape[1]*ncomp1 - 1 
    k2 = params2.shape[1]*ncomp2 - 1
    
    var1 = (ll1 -ll2).std()
    test1 = (ll1.sum() - ll2.sum() - k1 + k2)*nobs**(-.5)
    test1 = test1/var1
    p1 = 1 - stats.t.cdf(np.abs(test1),df=(nobs-k1-k2)) + stats.t.cdf(-np.abs(test1),df=(nobs-k1-k2))
    
    var2 =  ((ll1 - ll2)**2).mean()**.5
    test2 = (ll1.sum() - ll2.sum() - k1 + k2 )*nobs**(-.5)
    test2 = test2/var2
    p2 = 1 - stats.t.cdf(np.abs(test2),df=(nobs-k1-k2)) + stats.t.cdf(-np.abs(test2),df=(nobs-k1-k2))
    
    f.write(('\\begin{center}  \n'+
            '\\begin{tabular}{lclc} \n'+
            '\\toprule \n'+
            '\\textbf{Dep. Variable:} & %s & \\textbf{  R-squared: } &  %s \\\\ \n'%(ylabel, np.round(r2,3))  ))
    
    f.write(('\\textbf{No. Observations:} & %s & \\textbf{ Log-Likelihood:} & %s \\\\ \n'%(nobs,
                                                                                          np.round(ll.sum(),1))+
            '\end{tabular} \n'))
    
    
    
    #f.write('\end{tabular} \n'+
    #        '\end{center}\n')
    #f.close()
    
    return test1, test2, p1, p2

print nonnested_test(est2,est1)

(8.897585376501732, 8.79719221044862, 4.422054637877961e-19, 1.0663536269065003e-18)
